In [ ]:
from datasets import load_dataset,Audio
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load
def evaluateModel(path):
    common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "yue", split="test")
    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
    common_voice = common_voice.select(range(10))

    processor = WhisperProcessor.from_pretrained(path)
    model = WhisperForConditionalGeneration.from_pretrained(path).to("cuda")



    def map_to_pred(batch):
        audio = batch["audio"]
        input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
        batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

        with torch.no_grad():
            predicted_ids = model.generate(input_features.to("cuda"))[0]
        transcription = processor.decode(predicted_ids)
        batch["prediction"] = processor.tokenizer._normalize(transcription)
        return batch

    result = common_voice.map(map_to_pred)

    wer = load("wer")
    print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))


In [3]:
whisper_path=["openai/whisper-tiny","openai/whisper-base","openai/whisper-small","openai/whisper-medium","openai/whisper-large","openai/whisper-large-v2","openai/whisper-large-v3"]

In [10]:
evaluateModel(whisper_path[0])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

600.0


In [9]:
evaluateModel(whisper_path[1])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

453.8461538461538


In [8]:
evaluateModel(whisper_path[2])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

453.8461538461538


In [7]:
evaluateModel(whisper_path[3])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

600.0


In [6]:
evaluateModel(whisper_path[4])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

592.3076923076924


In [5]:
evaluateModel(whisper_path[5])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

523.0769230769231


In [4]:
evaluateModel(whisper_path[6])

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/5 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Reading metadata...: 2877it [00:00, 124571.71it/s]


Generating validation split: 0 examples [00:00, ? examples/s]

...

Reading metadata...: 25100it [00:00, 139660.64it/s]


Generating invalidated split: 0 examples [00:00, ? examples/s]

Reading metadata...: 1559it [00:00, 124962.64it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

153.84615384615387
